# Autocomplete Function With Tries

In [4]:
class Node:
    """This class represents one node of a trie tree.
    """

    def __init__(self, letter):
        self.letter = letter   # initialize letters in inserted word(s)
        self.isEnd = False     # tells whether end of word
        self.children = {}     # initialize dictionary of child nodes
        self.counter = 0       # initialize to count insertions of a word
        self.parent = None     # initialize parent node
        
        
class Trie:
    """This class represents the entirety of a trie tree.
    
    Methods
    -------
    insert(self, word)
        Inserts a word into the trie, creating nodes as required.
    lookup(self, word)
        Determines whether a given word is present in the trie.
    """
    def __init__(self, word_list = None):
        """Creates the Trie instance, inserts initial words if provided.
        
        Parameters
        ----------
        word_list : list
            List of strings to be inserted into the trie upon creation.
        """
        self.root = Node("")   # initialize root based on Node class
        
        if word_list != None:   # input all words from word list (wordbank)
            for word in word_list:    
                self.insert(word)
    
    
    def insert(self, word):
        """Inserts a word into the trie, creating missing nodes on the go.
        
        Parameters
        ----------
        word : str
            The word to be inserted into the trie.
        """
        # set current node as root
        currentNode = self.root 
        # remove case-sensitivity
        word = word.lower()   
        
        # insert letter into trie if not already inside
        for letter in word:
            if letter not in currentNode.children:
                currentNode.children[letter] = Node(letter)   # use Node 
            currentNode.children[letter].parent = currentNode    # set parent 
            currentNode = currentNode.children[letter]
        
        # mark end of word after inserting all letters into trie
        currentNode.isEnd = True
        
        # increase counter (1 for each insertion of word)
        currentNode.counter += 1
            

    def lookup(self, word):
        """Determines whether a given word is present in the trie.
        
        Parameters
        ----------
        word : str
            The word to be looked-up in the trie.
            
        Returns
        -------
        bool
            True if the word is present in trie; False otherwise.
        """
        # if word is blank, is is in the trie
        if word == "":
            return True
        
        # set current node as root
        currentNode = self.root
        # remove case-sensitivity
        word = word.lower()   
        
        # if each letter in word isn't in trie, word is not in trie
        for letter in word:
            if letter not in currentNode.children: # letter not in trie
                return False
            currentNode = currentNode.children[letter] # letter in trie
        return currentNode.isEnd
    
    
    def preorder_traversal(self):
        """Delivers the content of the trie in alphabetical order.
        
        Returns
        ----------
        list
            List of strings, all words from the trie in alphabetical order.
        """
        
        def preorder_sort(node):
            
            # base case when node = root (begin at root for preordered traversal)
            if node.children == {}:
                return node.letter
            
            # create empty list to populate with words
            preorderWordList = []
            
            # if end of word, append letter on node
            if node.isEnd:
                preorderWordList.append(node.letter)
                
            # traverse through each child, recursively calling the function... 
            # to add each word to the sorted list of words
            # sort keys with sorted() function
            for key in sorted(node.children.keys()):   
                preorderWordList.extend(preorder_sort(node.children[key]))
            
            # for each sorted node, add letters until all words are represented
            for letter in range(len(preorderWordList)):
                # don't include duplicates
                if (preorderWordList[letter] == node.letter) and \
                (node.isEnd == True):
                    continue
                preorderWordList[letter] = node.letter + \
                preorderWordList[letter]
            
            return preorderWordList

        
        # apply sort function to root, intiating recursion to include... 
        # all words in the trie
        return preorder_sort(self.root)

    
    def common_words(self, currentNode, firstTime = True, allWords = []): 
        """Recursively finds all common words inserted...
        into the trie and their counts. 

        Parameters
        ----------
        currentNode : object
        firstTime : bool
            Determines whether or not the function has already run.
        allWords : list
            List of tuples of words and their respective counts.

        Returns
        -------
        list
            List of tuples of words and their respective counts.
        """
        # reset the list for each new speaker
        if firstTime == True: 
            allWords = []
        
        # if currentNode has children, build words from the bottom up 
        # keep track of how many times each word is built
        if currentNode.children != {}:  
            for i in currentNode.children.keys():
                if currentNode.children[i].isEnd:
                    word = ""   # initialize word
                    letter = currentNode.children[i]   # last letter
                    count = letter.counter   # count every end of word                    
                    while letter != self.root:
                        # build word from last letter up
                        word = letter.letter + word   
                        letter = letter.parent

                    # save words and word counts in a list of tuples
                    allWords.append((word, count))
                
                # recurse until all words are counted
                self.common_words(currentNode.children[i], False, allWords)

        return allWords
 

    def k_most_common(self, k):
        """Finds k words inserted into the trie most often.
        
        Parameters
        ----------
        k : int
            Number of most common words to be returned.

        Returns
        ----------
        list
            List of tuples.
            
            Each tuple entry consists of the word and its frequency.
            The entries are sorted by frequency.

        Example
        -------
        >>> print(trie.k_most_common(3))
        [(‘the’, 154), (‘a’, 122), (‘i’, 122)]
        
        This means that the word ‘the’ has appeared 154 times.
        The second and third most common words both appeared 122 times.
        """
        # get common words in form of list of tuples from common_words() 
        commonWords = self.common_words(self.root)
        
        # convert list of tuples back to dictionary 
        countedWords = {}
        for element in commonWords:
            keys, x = element 
            countedWords[keys] = x 
        
        # sort words in descending order based on number of times they are counted 
        descendingCountedWords = sorted(countedWords.items(), \
                                        key = lambda t:(-t[1],t[0]))

        # return only the first k items in list
        return descendingCountedWords[0:k]
    
    
    def depth_first(self, currentNode, prefix):
        """Uses depth-first traversal to get all possible 
        solutions to the prefix. It populates the solutions list 
        with tuples including the possible solutions and their frequencies.
        
        Parameters
        ----------
        currentNode: object
        prefix : str
        
        """
        
        # if last node in word, add entire word to the solutions list
        # also include the frequency (word count)
        if currentNode.isEnd:
            self.solutions.append((prefix + currentNode.letter, \
                                   currentNode.counter))
        
        # use recursion to find every possible solution
        for child in currentNode.children.values():
            self.depth_first(child, prefix + currentNode.letter)
        
        
    def autocomplete(self, prefix):
        """Finds the most common word with the given prefix.

        You might want to reuse some functionality or ideas from Q4.

        Parameters
        ----------
        prefix : str
            The word part to be “autocompleted”.

        Returns
        ----------
        str
            The complete, most common word with the given prefix.
            
            The return value is equal to prefix if there... 
            is no valid word in the trie. The return value is also...
            equal to prefix if prefix is the most common word.
        """
        
        currentNode = self.root
        self.solutions = []
        
        # check to see if the prefix is in the trie
        for letter in prefix:
            if letter in currentNode.children:
                currentNode = currentNode.children[letter]
            else:
                return prefix   # if prefix not in tree, return prefix
        
        # use depth-first traversal to get all possible solutions
        self.depth_first(currentNode, prefix[:-1])

        # sort possible in descending order by frequency
        sorted_solutions = sorted(self.solutions, key=lambda \
                                  prefix:prefix[1], reverse=True)
        
        # return the most frequent word that matches (best match)
        return sorted_solutions[0][0]


In [5]:
import urllib.request
response = urllib.request.urlopen('http://bit.ly/CS110-Shakespeare')
bad_chars = [';', ',', '.', '?', '!', '1', '2', '3', '4', '5', '6', '7', '8', '9', '0', '_', '[', ']']

trieSH = Trie()

for line in response:
    line = line.decode(encoding = 'utf-8')
    line = filter(lambda i: i not in bad_chars, line)
    words = "".join(line).split()
    for word in words:
        trieSH.insert(word)

assert trieSH.autocomplete('hist') == 'history'
assert trieSH.autocomplete('en') == 'enter'
assert trieSH.autocomplete('cae') == 'caesar'
assert trieSH.autocomplete('gen') == 'gentleman'
assert trieSH.autocomplete('pen') == 'pen'
assert trieSH.autocomplete('tho') == 'thou'
assert trieSH.autocomplete('pent') == 'pentapolis'
assert trieSH.autocomplete('petr') == 'petruchio'
assert trieSH.autocomplete('CS') == 'CS'
assert trieSH.autocomplete('the') == 'the'
assert trieSH.autocomplete('Where is Mont') == 'Where is Mont'

print("Done")

Done
